In [90]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
#import winsound
import lightgbm as lgb

from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss, brier_score_loss
from sklearn.ensemble import RandomForestClassifier
import lightgbm
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [91]:
submit_prediction = True

In [105]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")


In [93]:
df_train

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.300
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.700
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.000
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.900
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
629995,629995,18,female,b.tech,4.86,70.7,yes,4.1,good,mixed,high,moderate,69.500
629996,629996,21,female,ba,7.08,54.4,yes,4.5,average,mixed,low,moderate,78.900
629997,629997,24,male,bca,0.64,44.2,yes,4.3,poor,online videos,low,moderate,19.599
629998,629998,20,male,b.com,1.54,75.1,yes,8.2,average,group study,high,moderate,59.100


In [94]:
df_test

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,630000,24,other,ba,6.85,65.2,yes,5.2,poor,group study,high,easy
1,630001,18,male,diploma,6.61,45.0,no,9.3,poor,coaching,low,easy
2,630002,24,female,b.tech,6.60,98.5,yes,6.2,good,group study,medium,moderate
3,630003,24,male,diploma,3.03,66.3,yes,5.7,average,mixed,medium,moderate
4,630004,20,female,b.tech,2.03,42.4,yes,9.2,average,coaching,low,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...
269995,899995,21,other,b.com,2.55,82.3,yes,8.4,average,mixed,medium,hard
269996,899996,17,female,b.com,0.49,46.4,yes,8.8,good,mixed,low,easy
269997,899997,22,male,bba,6.62,74.7,yes,5.5,good,coaching,high,easy
269998,899998,22,other,ba,4.08,51.8,yes,8.7,poor,online videos,high,moderate


In [103]:
# find missing values
missing_values = df_train.isnull().sum()
missing_values = missing_values[missing_values > 0]

if not missing_values.empty:
    plt.figure(figsize=(10, 6))
    sns.barplot(x=missing_values.index, y=missing_values.values, palette='viridis')
    plt.xticks(rotation=90)
    plt.xlabel('Features')
    plt.ylabel('Missing Values')
    plt.title('Missing Values per Feature')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found in the dataset.")

✅ No missing values found in the dataset.


In [106]:
#implementation of new features

diff_value_map = {"easy": 1, "moderate": 2, "hard": 3}

df_train["difficulty_score"] = (
    df_train["exam_difficulty"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map(diff_value_map)
)

df_train["class_attendance"] = df_train["class_attendance"] / 10
df_test["class_attendance"] = df_test["class_attendance"] / 10

df_train["study_hours_per_age"] = df_train["study_hours"] / df_train["age"]
df_train["effective_study"] = df_train["study_hours"] * df_train["class_attendance"] / 100
df_train["difficulty_adjusted_study"] = df_train["study_hours"] / df_train["difficulty_score"]
df_train["attendance_vs_difficulty"] = df_train["class_attendance"] / df_train["difficulty_score"]


df_test["difficulty_score"] = (
    df_test["exam_difficulty"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map(diff_value_map)
)

df_test["study_hours_per_age"] = df_test["study_hours"] / df_test["age"]
df_test["effective_study"] = df_test["study_hours"] * df_test["class_attendance"] / 100
df_test["difficulty_adjusted_study"] = df_test["study_hours"] / df_test["difficulty_score"]
df_test["attendance_vs_difficulty"] = df_test["class_attendance"] / df_test["difficulty_score"]



df_test = df_test.drop(["exam_difficulty"], axis=1)
df_train = df_train.drop(["exam_difficulty"], axis=1)

In [107]:
df_train

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_score,difficulty_score,study_hours_per_age,effective_study,difficulty_adjusted_study,attendance_vs_difficulty
0,0,21,female,b.sc,7.91,9.88,no,4.9,average,online videos,low,78.300,1,0.376667,0.781508,7.910,9.880
1,1,18,other,diploma,4.95,9.48,yes,4.7,poor,self-study,medium,46.700,2,0.275000,0.469260,2.475,4.740
2,2,20,female,b.sc,4.68,9.26,yes,5.8,poor,coaching,high,99.000,2,0.234000,0.433368,2.340,4.630
3,3,19,male,b.sc,2.00,4.95,yes,8.3,average,group study,high,63.900,2,0.105263,0.099000,1.000,2.475
4,4,23,male,bca,7.65,8.69,yes,9.6,good,self-study,high,100.000,1,0.332609,0.664785,7.650,8.690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629995,629995,18,female,b.tech,4.86,7.07,yes,4.1,good,mixed,high,69.500,2,0.270000,0.343602,2.430,3.535
629996,629996,21,female,ba,7.08,5.44,yes,4.5,average,mixed,low,78.900,2,0.337143,0.385152,3.540,2.720
629997,629997,24,male,bca,0.64,4.42,yes,4.3,poor,online videos,low,19.599,2,0.026667,0.028288,0.320,2.210
629998,629998,20,male,b.com,1.54,7.51,yes,8.2,average,group study,high,59.100,2,0.077000,0.115654,0.770,3.755


In [83]:
#Train data
categorical_columns = df_train.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(df_train[categorical_columns])

one_hot_df_train = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

df_train_encoded = pd.concat([df_train, one_hot_df_train], axis=1)

df_train_encoded = df_train_encoded.drop(categorical_columns, axis=1)


#Test data
categorical_columns = df_test.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(df_test[categorical_columns])

one_hot_df_test = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

df_test_encoded = pd.concat([df_test, one_hot_df_test], axis=1)

df_test_encoded = df_test_encoded.drop(categorical_columns, axis=1)


print(f"Encoded Train data : \n{df_train_encoded}")
print(f"Encoded Test data : \n{df_test_encoded}")

Encoded Train data : 
            id  age  study_hours  class_attendance  sleep_hours  exam_score  \
0            0   21         7.91              98.8          4.9      78.300   
1            1   18         4.95              94.8          4.7      46.700   
2            2   20         4.68              92.6          5.8      99.000   
3            3   19         2.00              49.5          8.3      63.900   
4            4   23         7.65              86.9          9.6     100.000   
...        ...  ...          ...               ...          ...         ...   
629995  629995   18         4.86              70.7          4.1      69.500   
629996  629996   21         7.08              54.4          4.5      78.900   
629997  629997   24         0.64              44.2          4.3      19.599   
629998  629998   20         1.54              75.1          8.2      59.100   
629999  629999   18         3.94              75.3          5.8      37.200   

        difficulty_score  stu

In [84]:
X = df_train_encoded.drop(columns=["exam_score"])  # remove target from traings data
y = df_train_encoded["exam_score"]

In [85]:
seeds = [i for i in range(6)]
models = []


if submit_prediction:
    kwargs = {
        "n_estimators": 5000,
        "learning_rate": 0.05,
        "max_depth": -1,          # often better than forcing 6
        "subsample": 0.8,
        "colsample_bytree": 0.8,  # 0.2 can be too restrictive
        "random_state": 42,
        "n_jobs": -1,
        "objective": "regression",
    }

    for s in seeds:
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=s
        )
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=s
        )

        model = lgb.LGBMRegressor(**kwargs)

        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse",
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
        )

        models.append(model)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004437 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1895
[LightGBM] [Info] Number of data points in the train set: 403200, number of used features: 33
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 62.494785
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003294 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1896
[LightGBM] [Info] Number of data points in the train set: 403200, number of used features: 3

In [86]:
if submit_prediction:
    preds = []
    for model in models:
        bi = getattr(model, "best_iteration", None)
        if bi is not None:
            preds.append(model.predict(df_test_encoded, num_iterations=bi))
        else:
            preds.append(model.predict(df_test_encoded))

    preds_arr = np.vstack(preds)
    mean_pred = preds_arr.mean(axis=0)

In [87]:
#submission file
submission = pd.DataFrame({
    "id": df_test["id"],
    "exam_score": mean_pred
})

submission.to_csv("submission.csv", index=False)

print("submission.csv created")

submission.csv created
